We have already seen this before, how to create **Data/Functional Attributes at run time**.

But now in this notebook I will show you one example of how to create and bind functional attribute at run time to an object, to get better & deeper understanding.

**Recap:**

In [9]:
class Person:
    pass

p1 = Person()
p2 = Person()

p1.name = 'Alex'
p1.__dict__, p2.__dict__

({'name': 'Alex'}, {})

In [10]:
p1.say_hello = lambda: 'Hello!'

# It will not become a method, it will be a simple function
p1.say_hello

<function __main__.<lambda>()>

And, now we already know that we can bind this function to a specific instance as well by using ```MethodType()```

**Let's take a look again**

In [11]:
def say_hello2(self):
    return f'{self.name} says hello'
say_hello(p1)

'Alex says hello'

In [12]:
from types import MethodType
func = MethodType(say_hello2, p1)

In [13]:
# func is a bounded method to an instance p1
func

<bound method say_hello2 of <__main__.Person object at 0x0750C4D0>>

In [15]:
# Still we can see it is not present in namespace. 
# and that's because we bind say_hello2 function to p1, but never assigned
# it to functional attribute of p1
p1.__dict__

{'name': 'Alex', 'say_hello': <function __main__.<lambda>()>}

In [16]:
# So lets go ahead and assign it
p1.say_hello2 = func

# Or we can do it in a sigle line as well.
# p1.say_hello2 = MethodType(say_hello2, p1)

In [17]:
p1.__dict__

{'name': 'Alex',
 'say_hello': <function __main__.<lambda>()>,
 'say_hello2': <bound method say_hello2 of <__main__.Person object at 0x0750C4D0>>}

In [18]:
from types import MethodType

class Person:
    def __init__(self, name):
        self.name = name
    
    def register_do_work(self, func):
        self._do_work = MethodType(func, self)
    
    def do_work(self):
        # we have done it, in this way because user can directly call do_work
        # without registering.
        do_work_method = getattr(self, '_do_work', None)
        
        if do_work_method:
            return do_work_method()
        else:
            raise AttributeError("You Must first register a do_work method.")
        

In [19]:
math_teacher = Person('Sarthak')
english_teacher = Person('John')

In [20]:
math_teacher.do_work()

AttributeError: You Must first register a do_work method.

In [21]:
def work_math(self):
    return f'{self.name} will teach Intergrals today'

math_teacher.register_do_work(work_math)

In [25]:
math_teacher.__dict__

{'name': 'Sarthak',
 '_do_work': <bound method work_math of <__main__.Person object at 0x074B0730>>}

In [22]:
math_teacher.do_work()

'Sarthak will teach Intergrals today'

In [23]:
def work_english(self):
    return f'{self.name} will teach Subject-Verb agreement today.'

english_teacher.register_do_work(work_english)

In [26]:
english_teacher.__dict__

{'name': 'John',
 '_do_work': <bound method work_english of <__main__.Person object at 0x074B08B0>>}

In [27]:
english_teacher.do_work()

'John will teach Subject-Verb agreement today.'

Isn't just awesome, that you can define different functionalities for ```_do_work()``` at runtime for each instances you created.

Observe that in this example, ```_do_work()``` for math_teacher has different functionality as of english_teacher and that's all defined at run time.

In [28]:
persons = [math_teacher, english_teacher]
for p in persons:
    print(p.do_work())

Sarthak will teach Intergrals today
John will teach Subject-Verb agreement today.


**This is really interesting as now we have different implementations for each instances and still we are calling them in a same way**